# Tarea 2
## Gabriel Daniel Aguilar Luna
### _Facultad de Ingeniería, Universidad Nacional Autónoma de México_
### _Ciudad de México, México_
#### gabriel.aguilar@ingenieria.unam.edu
***

In [ ]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
#Bibliotecas
from nltk.corpus import brown
from collections import Counter
from random import sample, choice
from itertools import chain
from copy import deepcopy
import cupy as cp
from scipy.special import softmax
from math import log

## Desarollo:

A partir del corpus seleccionado en la tarea anterior realizar un modelo del lenguaje neuronal con base en la arquitectura propuesta por Bengio (2003). El corpus ya debe estar preprocesado. S´ıganse los siguientes pasos:

In [ ]:
brown.sents()

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

 ## Insertar símbolos de inicio y final de cadena.

In [ ]:
# Etiquetas de inicio y final de cadena
BOS = '<bos>'
EOS = '<eos>'
bse = [[BOS]+sent+[EOS] for sent in brown.sents()]

In [ ]:
# Esta funcion devuelve una cadena conformada por las palabras en la lista
# de strings sent_list. Despues de cada palabra agrega un espacio ' '
# para facilitar la visualizacion.
# sent_list debe ser una lista de strings.
def sentString(sent_list):
    return ''.join(w+' ' for w in sent_list)[:-1]

In [ ]:
# Comprobamos que a las sentancias se les han agregado los
# simbolos de inicio BOS y final EOS.
sentString(sample(bse, 1)[0])

'<bos> Actually , however , this turns out to be only part of the picture . <eos>'

## Obtener los bigramas que aparecen en el texto (indexar numéricamente).

In [ ]:
# tokens es una lista de todas las cadenas de todas las sentencias en el corpus.
tokens = [w.lower() for w in chain(*[sent for sent in bse])]
# tipos es una lista de todas las palabras diferentes en el corpus
tipos = set(tokens)
# indice es un diccionario que asigna como valor un numero entero a cada tipo
# las cadenas del la lista tipos son las llaves del diccionario.
indice = dict(zip(tipos, range(len(tipos))))
# Se agrega el simbolo de <oov> para cadenas no vistas en el vocabulario
indice['<oov>'] = len(indice)

In [ ]:
# Caracteristicas del corpus y del indice de palabras
print("Tokens:\t\t",len(tokens))
print("Tipos:\t\t",len(tipos))
print("Indice:\t\t",len(indice))
print("Indice[BOS]:\t",indice[BOS])
print("Indice[EOS]:\t",indice[EOS])
print("Indice[OOV]:\t",indice['<oov>'])

Tokens:		 1275872
Tipos:		 49817
Indice:		 49818
Indice[BOS]:	 4734
Indice[EOS]:	 29696
Indice[OOV]:	 49817


In [ ]:
# La variable numSents es una lista de las sentencias del corpus representadas
# por los indices numericos de las cadenas que las conforman
numSents = deepcopy(bse)
for i in range(len(numSents)):
    for j in range(len(numSents[i])):
        numSents[i][j] = indice[numSents[i][j].lower()]

In [ ]:
# Se muestra la sentencia más corta:
min(numSents, key=len)

[4734, 16251, 29696]

Se crea una lista de bigramas numericos y otra con los bigramas string.

In [ ]:
bigramas = list(chain(*[zip(sent,sent[1:]) for sent in numSents]))
bigramas_str = list(chain(*[zip(sent,sent[1:]) for sent in bse]))

In [ ]:
# Algunos de los bigramas:
for muestra in range(50,54):
    print(bigramas[muestra],'\t',bigramas_str[muestra])

(3547, 6672) 	 ('the', 'praise')
(6672, 9062) 	 ('praise', 'and')
(9062, 36092) 	 ('and', 'thanks')
(36092, 8265) 	 ('thanks', 'of')


In [ ]:
# Total de bigramas:
len(bigramas)

1218532

##  Entrenar con los bigramas la red neuronal y obtener los valores para los hiperparámetros. Tomar de 100 unidades para la primera capa oculta (capa lineal) y 300 para la segunda capa oculta (capa con tanh).

Variables útiles:

In [ ]:
N = len(indice)
d = 100
m = 300
eta = 0.1
iters = 50

In [ ]:
# Esta funcion regresa el vector one hot para el indice dado
oneHots_array = cp.identity((N), dtype='uint8')
def get_oneHot(indi):
    return oneHots_array[indi,:]

Decidí crear una clase llamada red neuronal para trabajar, su función de init, una función para recorrer la red hacía adelante (forward), una función de entrenamiento (train), una de evaluación del modelo (eval) y una última que calcula la probabilidad de una cadena de entrada.

Manejé todos los cálculos como multiplicaciones de matrices.

In [ ]:
class red_neuronal:
    
    #inicializa la red
    def __init__(self, N_, d_, m_, eta_=0.1, iters_=100):
        self.N = N_
        self.d = d_
        self.m = m_
        self.eta = eta_
        self.iters = iters_        
        # Array correspondiente a la matriz C
        self.C = cp.random.rand(self.d,self.N)
        # Array correspondiente a la matriz W
        self.W = cp.random.rand(self.m,self.d)
        # Array correspondiente a la matriz U
        self.U = cp.random.rand(self.N,self.m)
    
    # Recorre la red hacia adelante
    # Retorna un arreglo unidimensional con las probabilidades para cada palabra.
    def forward(self,xi):
        # C(i) = Cx(i)
        self.Ci = cp.dot(self.C,xi)
        # h(i) = tanh(WC(i))
        self.hi = cp.tanh(cp.dot(self.W,self.Ci))
        # a(i) = Uh(i)
        self.a = cp.dot(self.U,self.hi)
        # softmax(a(i))
        return cp.array(softmax(self.a.get()))
    
    def train(self, bigrams):
        # Para cada iteración:
        for it in range(self.iters):
            # Variable auxiliar
            bi_n = 0
            # Para cada bigrama:
            for bi in bigrams:
                # Genera el vector de entrada
                xi = get_oneHot(bi[0])
                # Obtiene el vector de probabilidades P
                p = self.forward(xi)


                # Backpropagation

                # Al vector de probabilidades en la posicion j le resta 1
                p[bi[1]] -= 1
                self.d_out = p
                # d_h(k) = [1-h(i)^2] U.T d_out
                self.d_h = (1 - cp.square(self.hi))*cp.dot(self.U.T, self.d_out)
                # d_c = W.T d_h
                self.d_c = cp.dot(self.W.T, self.d_h)


                # Gradiente descendiente

                # U = U - eta d_out h(i)
                self.U -= self.eta * cp.dot(self.d_out[cp.newaxis].T, self.hi[cp.newaxis])
                # W = W - eta d_h C(i)
                self.W -= self.eta * cp.dot(self.d_h[cp.newaxis].T, self.Ci[cp.newaxis])
                # C = C - eta d_c x(i)
                self.C -= self.eta * cp.dot(self.d_c[cp.newaxis].T, xi[cp.newaxis])
                
            print('I'+str(it), end=(', ' if (it != iters-1) else '.'))
    
    def eval(self, bigrams):
        H = 0
        for bi in bigrams:
            xi = get_oneHot(bi[0])
            p = self.forward(xi)
            H -= log(p[bi[1]])
        print('Entropía:',H/len(bigrams), end='\n\n')
    
    def sentProba(self, sentencia):
        # Separa la sentencia por espacios
        sentencia = sentencia.split(' ')
        # Recorre cada token
        for w in range(len(sentencia)):
            # Si la sentencia se encuentra en el indice se le asigna el valor correspondiente
            # de lo contrario se le asigna el valor de <OOV>
            sentencia[w] = (indice[sentencia[w]] if (sentencia[w] in indice) else indice['<oov>'])
        bis = list(chain(*[zip(sentencia,sentencia[1:])]))
        proba = 1
        # Obtiene la probabilidad de la cadena
        for bi in bis:
            proba *= self.forward(get_oneHot(bi[0]))[bi[1]]
        return proba

In [ ]:
red = red_neuronal(N,d,m,eta,iters)

In [ ]:
red.train(bigramas[:10000])

I0, I1, I2, I3, I4, I5, I6, I7, I8, I9, I10, I11, I12, I13, I14, I15, I16, I17, I18, I19, I20, I21, I22, I23, I24, I25, I26, I27, I28, I29, I30, I31, I32, I33, I34, I35, I36, I37, I38, I39, I40, I41, I42, I43, I44, I45, I46, I47, I48, I49.

## Evaluar el modelo con Entropía

In [ ]:
red.eval(bigramas[10000:12000])

Entropía: 30.918138660756377



## Calcular la proabilidad de 5 oraciones no vistas en el entrenamiento.

In [ ]:
sent1 = '<BOS> Do not go gentle into that good night . <EOS>' # Dylan Thomas
sent2 = '<BOS> I am become death , the destroyer of worlds . <EOS>' # Robert Oppenheimer
sent3 = '<BOS> It was the best of times , it was the worst of times . <EOS>' # Charles Dickens
sent4 = '<BOS> Anything you can imagine you can make real . <EOS>' # Jules Verne
sent5 = '<BOS> Rage , rage against the dying of the light . <EOS>' # Dylan Thomas

In [ ]:
red.sentProba(sent1)

array(1.25246471e-126)

In [ ]:
red.sentProba(sent2)

array(4.07852619e-137)

In [ ]:
red.sentProba(sent3)

array(3.70811368e-195)

In [ ]:
red.sentProba(sent4)

array(4.2620401e-120)

In [ ]:
red.sentProba(sent5)

array(1.92471072e-140)

Podemos observar que la sentencia con la probabilidad más alta es la de Verne.

***

## Conclusiones
Aguilar Luna Gabriel Daniel:

Al realizar esta tarea me encontré con varios obstáculos, en un principio la ejecución tardaba demasiado, lo dejé correr durante unas 5 horas y según mis cálculos le harían falta unas 20 horas para recorrer el 70% de los bigramas generados, por lo que detuve la ejecución y me enfoque en mejorar el código lo más que pude, investigando me enteré de que era posible utilizar una GPU en Google Colaboratory así que cambie la biblioteca de Numpy por Cupy (Aunque al escribir el código me base en las funciones de Numpy) ya que lei que sus funciones esran idénticas por lo que solo hacia falta cambiar de biblioteca. Aún utilizando la GPU los tiempos de ejecución del entrenamiento eran muy largos, por lo que decidí reducir el numero de ejecuciones a 50 y el numero de bigramas para el entrenamiento a 10,000 y el de evaluación a 12,000.

Probablemente no estoy realizando el Backpropagation y el Gradiente descendiente de la forma más efectiva. Decidí hacerlo de forma Matricial porque me paracío la forma más clara y sencilla de programarlo.

De cualquier forma, aunque no obtuve los mejores resultados está actividad me ha dejado comprender muy bien como funciona, por lo menos en lo general, una red neuronal.


***

## Fuentes de Consulta:

- [1] Numpy, "NumPy Reference — NumPy v1.21 Manual", Numpy.org, 2021. [Online]. Available: https://numpy.org/doc/stable/reference/index.html. [Accessed: 09- Aug- 2021].